In [1]:
import getpass
import os
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain_core.tools import tool

In [2]:
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')

In [3]:
llm = ChatGroq(temperature=0, model="llama3-70b-8192")

In [4]:
@tool
def complex_tool(int_arg: int, float_arg: float, dict_arg: dict) -> int:
    """Hacer algo complejo con una herramienta compleja."""
    return int_arg * float_arg

In [5]:
llm_with_tools = llm.bind_tools(
    [complex_tool],
)

In [12]:
# Define chain
chain = llm_with_tools | (lambda msg: msg.tool_calls[0]["args"]) | complex_tool

In [18]:
resultado = chain.invoke(
    "usa el complex tool. los args son 5, 2.1, diccionario vacío. no olvide dict_arg"
)
resultado

10.5

In [19]:
from typing import Any

from langchain_core.runnables import Runnable, RunnableConfig

In [20]:
def try_except_tool(tool_args: dict, config: RunnableConfig) -> Runnable:
    try:
        complex_tool.invoke(tool_args, config=config)
    except Exception as e:
        return f"Llamada a la herramienta con argumentos:\n\n{tool_args}\n\nAparece el siguiente error:\n\n{type(e)}: {e}"

In [23]:
chain = llm_with_tools | (lambda msg: msg.tool_calls[0]["args"]) | try_except_tool
chain

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001DB27C8EE90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001DB27D17460>, model_name='llama3-70b-8192', temperature=1e-08, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'complex_tool', 'description': 'complex_tool(int_arg: int, float_arg: float, dict_arg: dict) -> int - Hacer algo complejo con una herramienta compleja.', 'parameters': {'type': 'object', 'properties': {'int_arg': {'type': 'integer'}, 'float_arg': {'type': 'number'}, 'dict_arg': {'type': 'object'}}, 'required': ['int_arg', 'float_arg', 'dict_arg']}}}]})
| RunnableLambda(lambda msg: msg.tool_calls[0]['args'])
| RunnableLambda(try_except_tool)

In [26]:
resultado = chain.invoke(
    "usa el try_except_tool. los args son 5, 2.1, diccionario vacío. no olvide dict_arg"
)
print(resultado)

None
